In [0]:
%sql
-- Create SQL Procedure for sp_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING 
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING'
AS
BEGIN
  DECLARE my_log_id STRING;
  CALL gap_catalog.log.start_performance_test(
      test_case_name => 'ADS_SMA_PROC',
      step_name => 'ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_OPT',
      psource_sys_origin  => 'BUSY_BANKING_OPT',
      process_key => p_process_key,
      target_table => 'CASE_PHASE_PROPERTIES',
      log_id => my_log_id,
      p_load_date => p_load_date );
  
--SET use_cached_result = false;
truncate table gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING;

INSERT INTO gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING 
      SELECT
          STG_SMA.CPPROP_KEY,
          STG_SMA.CPPROP_SOURCE_ID,
          STG_SMA.CPPROP_SOURCE_SYSTEM_ID,
          STG_SMA.CPPROP_SOURCE_SYS_ORIGIN,
          STG_SMA.CPPROP_PARENT_KEY,
          -1 AS CASEPH_KEY,
          STG_SMA.CPPTP_KEY,
          NVL(CAS.CTP_KEY, -1) AS CTP_KEY,
          STG_SMA.CASE_KEY,
          NVL(CAS.CASE_START_DATE, TO_DATE('01011000', 'ddMMyyyy')) AS CASE_START_DATE,
          STG_SMA.CPPROP_VALUE_TEXT,
          STG_SMA.CPPROP_VALUE_DATE,
          STG_SMA.CPPROP_VALUE_NUMBER,
          -1 AS CPPRV_KEY
      FROM
          gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES STG_SMA
      LEFT JOIN
          gap_catalog.ads_owner.CASES CAS
      ON
          CAS.CASE_KEY = STG_SMA.CASE_KEY
          AND CAS.CTP_KEY = 1429
          AND CAS.CASE_VALID_TO = TO_DATE('01013000', 'ddMMyyyy')
      WHERE
          STG_SMA.CTP_KEY = 1429
;

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF;

CREATE TABLE gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      BIGINT,
  CPPROP_KEY  BIGINT,
  --CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH ' || max_key || ' INCREMENT BY 1),
  CPPROP_SOURCE_ID  string,
  CPPROP_SOURCE_SYSTEM_ID  string,
  CPPROP_SOURCE_SYS_ORIGIN  string,
  CPPROP_PARENT_KEY  BIGINT,
  CASEPH_KEY  BIGINT,
  CPPTP_KEY  BIGINT,
  CTP_KEY  BIGINT,
  CASE_KEY  BIGINT,
  CASE_START_DATE  DATE,
  CPPROP_VALUE_TEXT  string,
  CPPROP_VALUE_DATE  DATE,
  CPPROP_VALUE_NUMBER  BIGINT,
  CPPRV_KEY  BIGINT)
USING DELTA
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true'
);


INSERT INTO gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
SELECT 
    'N' AS tech_del_flg, 
    CASE WHEN trg.CPPROP_KEY IS NULL THEN 'Y' ELSE 'N' END AS tech_new_rec,
    trg.rid AS tech_rid,
    src.CPPROP_KEY, 
    src.CPPROP_SOURCE_ID, 
    src.CPPROP_SOURCE_SYSTEM_ID, 
    src.CPPROP_SOURCE_SYS_ORIGIN, 
    src.CPPROP_PARENT_KEY, 
    src.CASEPH_KEY, 
    src.CPPTP_KEY, 
    src.CTP_KEY, 
    src.CASE_KEY, 
    src.CASE_START_DATE, 
    src.CPPROP_VALUE_TEXT, 
    src.CPPROP_VALUE_DATE, 
    src.CPPROP_VALUE_NUMBER, 
    src.CPPRV_KEY
FROM 
    (SELECT 
        CPPROP_KEY, 
        CPPROP_SOURCE_ID, 
        CPPROP_SOURCE_SYSTEM_ID, 
        CPPROP_SOURCE_SYS_ORIGIN, 
        CPPROP_PARENT_KEY, 
        CASEPH_KEY, 
        CPPTP_KEY, 
        CTP_KEY, 
        CASE_KEY, 
        CASE_START_DATE, 
        CPPROP_VALUE_TEXT, 
        CPPROP_VALUE_DATE, 
        CPPROP_VALUE_NUMBER, 
        CPPRV_KEY
     FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING xc
     WHERE (CPPROP_SOURCE_SYSTEM_ID = 'SMA'
            AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
            AND CTP_KEY = 1429
            AND CPPTP_KEY IN (
                SELECT DISTINCT CPPTP_KEY 
                FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING
            ))
    ) src
LEFT JOIN
    (SELECT  
        cpprop_key AS rid, 
        t.* 
     FROM  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES t
     WHERE CPPROP_CURRENT_FLAG = 'Y'
       AND CPPROP_VALID_TO = to_date('01013000','ddMMyyyy')
       AND CPPROP_SOURCE_SYSTEM_ID = 'SMA'
       AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
       AND CTP_KEY = 1429
       AND CPPTP_KEY IN (
           SELECT DISTINCT CPPTP_KEY 
           FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING
       )
    ) trg 
ON trg.CPPROP_KEY = src.CPPROP_KEY
   AND trg.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd') 
WHERE (
    NOT (src.CPPROP_SOURCE_ID <=> trg.CPPROP_SOURCE_ID) OR
    NOT (src.CPPROP_SOURCE_SYSTEM_ID <=> trg.CPPROP_SOURCE_SYSTEM_ID) OR
    NOT (src.CPPROP_SOURCE_SYS_ORIGIN <=> trg.CPPROP_SOURCE_SYS_ORIGIN) OR
    NOT (src.CPPROP_PARENT_KEY <=> trg.CPPROP_PARENT_KEY) OR
    NOT (src.CASEPH_KEY <=> trg.CASEPH_KEY) OR
    NOT (src.CPPTP_KEY <=> trg.CPPTP_KEY) OR
    NOT (src.CTP_KEY <=> trg.CTP_KEY) OR
    NOT (src.CASE_KEY <=> trg.CASE_KEY) OR
    NOT (src.CASE_START_DATE <=> trg.CASE_START_DATE) OR
    NOT (src.CPPROP_VALUE_TEXT <=> trg.CPPROP_VALUE_TEXT) OR
    NOT (src.CPPROP_VALUE_DATE <=> trg.CPPROP_VALUE_DATE) OR
    NOT (src.CPPROP_VALUE_NUMBER <=> trg.CPPROP_VALUE_NUMBER) OR
    NOT (src.CPPRV_KEY <=> trg.CPPRV_KEY) OR
    trg.CPPROP_KEY IS NULL OR 
    trg.CPPROP_DELETED_FLAG = 'Y'
);

MERGE INTO gap_catalog.ads_owner.CASE_PHASE_PROPERTIES AS tgt
USING (
  SELECT 
    d.cpprop_key,
    d.ctp_key, 
    d.case_start_date, 
    d.CPPTP_KEY
  FROM gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF d
  WHERE d.tech_rid IS NOT NULL
) AS src
ON tgt.CPPROP_KEY = src.cpprop_key
   AND tgt.CTP_KEY = src.ctp_key
   AND tgt.CASE_START_DATE = src.case_start_date
   AND tgt.CPPTP_KEY = src.CPPTP_KEY
    AND tgt.CPPROP_CURRENT_FLAG = 'Y'
    AND tgt.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
WHEN MATCHED
THEN UPDATE SET
  CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
  CPPROP_UPDATE_PROCESS_KEY = p_process_key,
  CPPROP_CURRENT_FLAG = 'N',
  CPPROP_VALID_TO = to_date(p_load_date, 'yyyy-MM-dd') - 1;

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
    ( CPPROP_KEY, 
      CPPROP_SOURCE_ID, 
      CPPROP_SOURCE_SYSTEM_ID, 
      CPPROP_SOURCE_SYS_ORIGIN, 
      CPPROP_PARENT_KEY, 
      CASEPH_KEY, 
      CPPTP_KEY, 
      CTP_KEY, 
      CASE_KEY, 
      CASE_START_DATE, 
      CPPROP_VALUE_TEXT, 
      CPPROP_VALUE_DATE, 
      CPPROP_VALUE_NUMBER, 
      CPPROP_VALID_FROM, 
      CPPROP_VALID_TO,
      CPPROP_CURRENT_FLAG, 
      CPPROP_DELETED_FLAG, 
      CPPROP_INSERTED_DATETIME, 
      CPPROP_INSERT_PROCESS_KEY, 
      CPPROP_UPDATED_DATETIME, 
      CPPROP_UPDATE_PROCESS_KEY, 
      CPPRV_KEY)
    select CPPROP_KEY, 
      CPPROP_SOURCE_ID, 
      CPPROP_SOURCE_SYSTEM_ID, 
      CPPROP_SOURCE_SYS_ORIGIN, 
      CPPROP_PARENT_KEY, 
      CASEPH_KEY, 
      CPPTP_KEY, 
      CTP_KEY, 
      CASE_KEY, 
      CASE_START_DATE, 
      CPPROP_VALUE_TEXT, 
      CPPROP_VALUE_DATE, 
      CPPROP_VALUE_NUMBER, 
      to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
      to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
      'Y' as CPPROP_CURRENT_FLAG, 
      tech_del_flg as CPPROP_DELETED_FLAG, 
      CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
      p_process_key as CPPROP_INSERT_PROCESS_KEY, 
      CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
      p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
      CPPRV_KEY
      from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
    where tech_new_rec = 'N';

    insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
    ( CPPROP_KEY, 
      CPPROP_SOURCE_ID, 
      CPPROP_SOURCE_SYSTEM_ID, 
      CPPROP_SOURCE_SYS_ORIGIN, 
      CPPROP_PARENT_KEY, 
      CASEPH_KEY, 
      CPPTP_KEY, 
      CTP_KEY, 
      CASE_KEY, 
      CASE_START_DATE, 
      CPPROP_VALUE_TEXT, 
      CPPROP_VALUE_DATE, 
      CPPROP_VALUE_NUMBER, 
      CPPROP_VALID_FROM, 
      CPPROP_VALID_TO,
      CPPROP_CURRENT_FLAG, 
      CPPROP_DELETED_FLAG, 
      CPPROP_INSERTED_DATETIME, 
      CPPROP_INSERT_PROCESS_KEY, 
      CPPROP_UPDATED_DATETIME, 
      CPPROP_UPDATE_PROCESS_KEY, 
      CPPRV_KEY)
    select 
      xxhash64(uuid()) AS CPPROP_KEY, 
      CPPROP_SOURCE_ID, 
      CPPROP_SOURCE_SYSTEM_ID, 
      CPPROP_SOURCE_SYS_ORIGIN, 
      CPPROP_PARENT_KEY, 
      CASEPH_KEY, 
      CPPTP_KEY, 
      CTP_KEY, 
      CASE_KEY, 
      CASE_START_DATE, 
      CPPROP_VALUE_TEXT, 
      CPPROP_VALUE_DATE, 
      CPPROP_VALUE_NUMBER, 
      to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
      to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
      'Y' as CPPROP_CURRENT_FLAG, 
      tech_del_flg as CPPROP_DELETED_FLAG, 
      CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
      p_process_key as CPPROP_INSERT_PROCESS_KEY, 
      CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
      p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
      CPPRV_KEY
      from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
    where tech_new_rec = 'Y';

  CALL gap_catalog.log.complete_performance_test(
        plog_id => my_log_id);
  
END;